In [114]:
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support


In [115]:
df = pd.read_csv('spam.csv', encoding='ISO-8859-1')
le = LabelEncoder()
df.head()


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [116]:
data = df.values
data[6]

array(['ham',
       'Even my brother is not like to speak with me. They treat me like aids patent.',
       nan, nan, nan], dtype=object)

In [117]:
X = data[:, 1]
y = data[:, 0]



In [118]:
X.shape, y.shape

((5572,), (5572,))

In [119]:
tokenizer = RegexpTokenizer('\w+')
sw = set(stopwords.words('english'))
ps = PorterStemmer()

In [120]:
def getStem(review):
    review = review.lower()
    tokens = tokenizer.tokenize(review) # breaking into small words
    removed_stopwords = [w for w in tokens if w not in sw]
    stemmed_words = [ps.stem(token) for token in removed_stopwords]
    clean_review = ' '.join(stemmed_words)
    return clean_review

In [121]:
# get a clean document
def getDoc(document):
    d = []
    for doc in document:
        d.append(getStem(doc))
    return d

In [122]:
stemmed_doc = getDoc(X)

In [123]:
stemmed_doc[:10]

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri 2 wkli comp win fa cup final tkt 21st may 2005 text fa 87121 receiv entri question std txt rate c appli 08452810075over18',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl 3 week word back like fun still tb ok xxx std chg send ã â 1 50 rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press 9 copi friend callertun',
 'winner valu network custom select receivea ã â 900 prize reward claim call 09061701461 claim code kl341 valid 12 hour',
 'mobil 11 month u r entitl updat latest colour mobil camera free call mobil updat co free 08002986030']

In [124]:
cv = CountVectorizer()

In [125]:
# create my vocab
vc = cv.fit_transform(stemmed_doc)

In [126]:
X = vc.todense()

In [147]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

In [148]:
# NB from sklearn

In [188]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


In [178]:
model = MultinomialNB()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.977705274605764

In [179]:
messages = [
    """
    Hi Kunal,
We invite you to participate in MishMash - India’s largest online diversity hackathon. 
The hackathon is a Skillenza initiative and sponsored by Microsoft, Unity, Unilever, Gojek, Rocketium and Jharkhand Government. 
We have a special theme for you - Deep Tech/Machine Learning - sponsored by Unilever, which will be perfect for you.
    """,
    """Join us today at 12:00 PM ET / 16:00 UTC for a Red Hat DevNation tech talk on AWS Lambda and serverless Java with Bill Burke.
Have you ever tried Java on AWS Lambda but found that the cold-start latency and memory usage were far too high? 
In this session, we will show how we optimized Java for serverless applications by leveraging GraalVM with Quarkus to 
provide both supersonic startup speed and a subatomic memory footprint.""",

    """We really appreciate your interest and wanted to let you know that we have received your application.
There is strong competition for jobs at Intel, and we receive many applications. As a result, it may take some time to get back to you.
Whether or not this position ends up being a fit, we will keep your information per data retention policies, 
so we can contact you for other positions that align to your experience and skill set.
"""
,
"""
 Massive Discounts Delivered!
 Hi  Priyanshi,
 Your Massive Discount is here!With just 1 day left, we wouldn't want you to miss this opportunity!75% OFF on Online Courses50% OFF on CB Accelerate Live Interactive Courses
 Hurry, Only 1 Day Left
"""
]

In [180]:
def prepare(messages):
    d = getDoc(messages)
    # dont do fit_transform!! it will create new vocab.
    return cv.transform(d)

messages = prepare(messages)

In [181]:
y_pred = model.predict(messages)
y_pred

array(['ham', 'spam', 'ham', 'ham'], dtype='<U4')

In [182]:
l=[]
for i in y_test:
    if i=="ham":
        l.append(0)
    else:
        l.append(1)

k=len(y_test)
k    

1839

In [183]:
m=[]

for i in y_train[:k]:
    if i=="ham":
        m.append(0)
    else:
        m.append(1)


In [184]:
precision_recall_fscore_support(m,l, average=None)


(array([0.8720857, 0.1468254]),
 array([0.86554096, 0.15416667]),
 array([0.868801 , 0.1504065]),
 array([1599,  240], dtype=int64))

In [185]:
f1_score(m,l, average=None)



array([0.868801 , 0.1504065])

In [186]:
precision_score(m, l, average=None)


array([0.8720857, 0.1468254])

In [189]:
recall_score(m,l,average=None)

array([0.86554096, 0.15416667])